In [ ]:
import pytorch_lightning as pl
import pytorch_lightning.loggers

import torch
import aegnn
from aegnn.models.networks import GraphRes
from scripts.train import LearningRateLogger
from aegnn.models.utils.map import compute_map
import os
import random
import shutil


#ncars alredy split
#https://drive.google.com/file/d/1vlByGVjqmyYvbzLSIzZzNLfcjfTJijyz/view


#save data in raw and make dirs train,validation,test

split=False

if(split):
    ### Copy the data into the train, validation and test directories (80%, 10%, 10%)
    dataset='ncaltech101'

    original_dir= os.getcwd()+'/data/'+dataset
    train_dir= original_dir +'/training'
    val_dir= original_dir +'/validation'
    test_dir= original_dir +'/test'

    original_dir+='/raw'

    ### Make a directory for each class in the train, validation, test directories.

    class_counts = {}

    class_folders = os.listdir(original_dir)

    for class_dir in class_folders:
        os.mkdir(os.path.join(train_dir, class_dir))
        os.mkdir(os.path.join(val_dir, class_dir))
        os.mkdir(os.path.join(test_dir, class_dir))
        
        class_counts[class_dir] = len(os.listdir(os.path.join(original_dir, class_dir)))

    print (class_counts)

    for class_dir in class_folders:
        images = os.listdir(os.path.join(original_dir, class_dir))
        random.seed(42)
        random.shuffle(images)

        train_images = images[:int(0.8 * class_counts[class_dir])]
        val_images = images[int(0.8 * class_counts[class_dir]):int(0.9 * class_counts[class_dir])]
        test_images = images[int(0.9 * class_counts[class_dir]):]

        for image in train_images:
            shutil.copyfile(os.path.join(original_dir, class_dir, image), os.path.join(train_dir, class_dir, image))

        for image in val_images:
            shutil.copyfile(os.path.join(original_dir, class_dir, image), os.path.join(val_dir, class_dir, image))

        for image in test_images:
            shutil.copyfile(os.path.join(original_dir, class_dir, image), os.path.join(test_dir, class_dir, image))

### Check that the data has been split correctly
for split in [train_dir, val_dir, test_dir]:
    total = 0
    for class_dir in os.listdir(split):
        total += len(os.listdir(os.path.join(split, class_dir)))
    print(split, total)







In [ ]:
!CUDA_VISIBLE_DEVICES=0 AEGNN_DATA_DIR=$(pwd)/data python scripts/preprocessing.py --dataset "ncars" --num-workers 8 --gpu 0

In [ ]:
!python evaluation/accuracy_per_events.py ~/home/ale/Downloads/GDL/data/log/checkpoints/ncars/recognition/20230504003225/epoch=49-step=5099.pt --device cuda --dataset ncars

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
model_name = "ncaltech101" #ncars
#model_name = "ncars"
file_path = f'/home/ale/Downloads/aegnn_results/accuracy_per_events_{model_name}.pkl'

with open(file_path, 'rb') as file:
    flops_data = pickle.load(file)
flops_df = pd.DataFrame(flops_data)
save_to_excel = True
if save_to_excel == True:    
    # Specify the output file path
    output_file_path = f'/home/ale/Downloads/aegnn_results/accuracy_per_events_{model_name}.xlsx'

    # Save the DataFrame to the Excel file
    flops_df.to_excel(output_file_path, index=False, engine='openpyxl')
    flops_df.info()
    x = flops_df["max_num_events"]
    y = flops_df["accuracy"]
    plt.plot(x, y)
    plt.xlabel("Number of Events")
    plt.ylabel("Recognition Accuracy")
    plt.title(f"Dataset: {model_name}")
    plt.grid()
    plt.savefig(f'/home/ale/Downloads/aegnn_results/accuracy_per_events_{model_name}.png')
    plt.show()
    
    

In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

file_path = '../aegnn_results/flops.pkl'

with open(file_path, 'rb') as file:
    flops_data = pickle.load(file)


flops_df = pd.DataFrame(flops_data)


# TODO: set save_to_excel = True if you run this block the first time 
save_to_excel = False
if save_to_excel == True:    
    # Specify the output file path
    output_file_path = 'results/flops_data_ncars.xlsx'

    # Save the DataFrame to the Excel file
    flops_df.to_excel(output_file_path, index=False, engine='openpyxl')

flops_df['Mflops/ev'] = flops_df['flops'] / 1e6 / 10000

total_df=flops_df[flops_df['layer'] == 'total']
print(total_df['layer'].value_counts()['total'])
total_df_dense = total_df[total_df['model'] == 'gnn_dense'].dropna(subset=['flops'])
total_df_ours = total_df[total_df['model'] == 'ours'].dropna(subset=['flops'])



average_total_dense= total_df_dense['flops'].sum()/1e6/10000
average_total_ours= total_df_ours['flops'].sum()/1e6/10000


print(average_total_dense)
print(average_total_ours)